Read data. A failed image is deleted and grouped by the number of ships.
/you can choose another criterion, such as mask weight/
validate images & masks is selected one picture cyclically from each group.

In [4]:
import math
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io as io
from tqdm import tqdm_notebook 

masks_train = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations_v2.csv')
masks_train = masks_train[masks_train['ImageId'] != '6384c3e78.jpg']
#https://www.kaggle.com/meaninglesslives/airbus-ship-detection-data-visualization
masks_train['ship_count'] = masks_train.groupby('ImageId')['ImageId'].transform('count')
masks_train.loc[masks_train['EncodedPixels'].isnull().values,'ship_count'] = 0  #see infocusp's comment
_masks_train = pd.concat([
                  masks_train[masks_train["ship_count"] == 15].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] == 14].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] == 13].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] == 12].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] == 11].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] == 10].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  9].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  8].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  7].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  6].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  5].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  4].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  3].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  2].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  1].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId"),
                  masks_train[masks_train["ship_count"] ==  0].reset_index().drop(columns=['index']).\
                              reset_index().set_index("ImageId") ]).\
    sort_values("index", inplace=False)
masks_train = _masks_train.drop(columns=['index']).reset_index()
del(_masks_train)
#print (masks_train.head())

train_dir="../input/airbus-ship-detection/train_v2/"
train_file_names = masks_train['ImageId'].unique()
train_file_names = train_file_names.tolist()
print("Train files :",len(train_file_names))


test_dir="../input/airbus-ship-detection/test_v2/"
test_file_names=os.listdir(test_dir)
print("Test files :",len(test_file_names))

train_len = len(train_file_names)
test_len = len(test_file_names)

o_size, w_size = 768, 256 # 384, 128, 64
ow_coeff = int(o_size / w_size)

if w_size*ow_coeff != o_size:
    print ('w_ o_ size error')
print (ow_coeff, ow_coeff*ow_coeff)


Train files : 192555
Test files : 15606
3 9


In [5]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def get_mask_img(ImageId):
    img_masks = masks_train['EncodedPixels'][masks_train['ImageId'] == ImageId].tolist()
    mask_img = np.zeros((768, 768))
    if len(img_masks)==1 and pd.isna(img_masks):
        return mask_img
    for mask in img_masks:
        mask_img += rle_decode(mask)
    return mask_img


In [6]:
f_val_len = 100 # depends on the power of your machine
#f_val_imgs = np.memmap('/data/ssd1/airbus_ship/data/airbus_imgs_val.dat', dtype='float32', mode='w+', \
#                   shape=(f_val_len*ow_coeff*ow_coeff, w_size, w_size, 3))
#f_val_masks = np.memmap('/data/ssd1/airbus_ship/data/airbus_masks_val.dat', dtype='float32', mode='w+', \
#                   shape=(f_val_len*ow_coeff*ow_coeff, w_size, w_size, 1))
f_val_imgs = np.zeros((f_val_len*ow_coeff*ow_coeff, w_size, w_size, 3), dtype='float32')
f_val_masks = np.zeros((f_val_len*ow_coeff*ow_coeff, w_size, w_size, 1), dtype='float32')
# make initial validate sequence
val_len = 0
for k in range(16):
    img = io.imread(train_dir+train_file_names[k])
    mask=get_mask_img(train_file_names[k])
    for i in range(ow_coeff):
        for j in range(ow_coeff):
            f_val_imgs[val_len,:,:,:] = img[w_size*i:w_size*i+w_size, w_size*j:w_size*j+w_size]/255.
            f_val_masks[val_len,:,:,0] = mask[w_size*i:w_size*i+w_size, w_size*j:w_size*j+w_size]
            val_len += 1
raw_image_i = k
    
print (val_len,raw_image_i)


144 15


we have the initial sequence obtained from 15 images to which we will add new ones.

load parts of keras and define a network

https://www.kaggle.com/cpmpml/fast-iou-metric-in-numpy-and-tensorflow

https://www.kaggle.com/shaojiaxin/u-net-with-simple-resnet-blocks

In [7]:
from keras.layers import Input, Dropout, Conv2D, MaxPooling2D, Activation
from keras.layers import BatchNormalization,Add,concatenate,Conv2DTranspose
from keras.layers import UpSampling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
#from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

import tensorflow as tf


def get_iou_vector(A, B):
    # Numpy version
    
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def my_iou_metric(label, pred):
    # Tensorflow version
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)

from keras.utils.generic_utils import get_custom_objects

get_custom_objects().update({'my_iou_metric': my_iou_metric })


Using TensorFlow backend.


In [8]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = Activation('relu')(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = Activation('relu')(blockInput)
    x = BatchNormalization()(x)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

# Build model
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16)
    convm = Activation('relu')(convm)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = Activation('relu')(uconv4)
    
    # 12 -> 25
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = Activation('relu')(uconv3)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = Activation('relu')(uconv2)
    
    # 50 -> 101
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = Activation('relu')(uconv1)
    
    uconv1 = Dropout(DropoutRatio/2)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

input_layer = Input((w_size, w_size, 3))
output_layer = build_model(input_layer, 16, 0.5)

model = Model(input_layer, output_layer)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[my_iou_metric])
#model.summary()



Here we take the next image and check how the network trained in the previous frames is able to predict the mask.
If the accuracy is acceptable, then this image(crop) is rejected, if the network could not calculate the mask, then this image(crop) is useful to be included in validate sequence.

In [ ]:
batch_size = ow_coeff*ow_coeff*2
precision = 0.85

#f_m_save = "./val_prep_0.model"
f_w_save = "../input/weights0/val_prep.weights" # my pretrained weights
model.load_weights(f_w_save, by_name=False)

K.set_value(model.optimizer.lr, 1e-4)
lr = K.get_value(model.optimizer.lr)

while True:
    print(time.ctime())
# network training in initial or added sequence
    fit = model.fit(f_val_imgs[:val_len], f_val_masks[:val_len],
                    batch_size=batch_size, 
                    epochs=1, 
                    verbose=2
                   )
    
    current_accu = fit.history['my_iou_metric'][0]
    current_loss = fit.history['loss'][0]
    if current_accu > precision:
#        model.save(f_m_save)
#        model.save_weights(f_w_save)
        while True:
            new_img_test = 0
            img = io.imread(train_dir+train_file_names[raw_image_i])
            mask=get_mask_img(train_file_names[raw_image_i])
            print ("ImageId",train_file_names[raw_image_i])
            raw_image_i += 1
            print ("next image",raw_image_i)
            for i in range(ow_coeff):
                for j in range(ow_coeff):
                    _img = img[w_size*i:w_size*i+w_size, w_size*j:w_size*j+w_size,:]/255.
                    _img = _img.reshape(1,w_size,w_size,3)
                    _mask = mask[w_size*i:w_size*i+w_size, w_size*j:w_size*j+w_size]
                    _mask = _mask.reshape(1,w_size,w_size,1)
                    
                    _pred = model.predict(_img)
                    val_iou = get_iou_vector(_mask, _pred > 0.5)
                    
                    if val_iou < precision*0.9:
                        # You can look at the images(crop) that is added
                        #
                        #fig, axes = plt.subplots(1, 3, figsize=(30, 10))
                        #axes[0].set_axis_off()
                        #axes[1].set_axis_off()
                        #axes[2].set_axis_off()
                        #axes[1].set_title(val_iou, fontsize=20)
                        #axes[0].imshow(_img.squeeze())
                        #axes[1].imshow(_pred.squeeze())
                        #axes[2].imshow(_mask.squeeze())
                        #
                        #plt.show(block=True)
                        new_img_test = 1
                        f_val_imgs[val_len,:,:,:] = _img[0,:,:,:]
                        f_val_masks[val_len,:,:,0] = _mask[0,:,:,0]
                        val_len += 1
                        print ("new validate lenght", val_len)
                        #f_val_imgs.flush()
                        #f_val_masks.flush()
# If the frame is added, then you need to clarify the network weights.
# If not then prepare next image
            if new_img_test > 0:
                break           

    if raw_image_i >= train_len:
        raw_image_i = 0
    if val_len >= f_val_len:
        break

#del(f_val_imgs)
#del(f_val_masks)
